In [ ]:

import pandas as pd    
import xgboost
from sklearn.model_selection import train_test_split
import time

import pandas as pd # dataframes
import numpy as np # algebra & calculus

import gc                         
gc.enable()       

import xgboost
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

In [ ]:
from zipfile import ZipFile
import os

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        with ZipFile(os.path.join(dirname, filename), 'r') as zip:
            zip.printdir()
            zip.extractall()
            
orders = pd.read_csv('/kaggle/working/orders.csv' )
order_products_train = pd.read_csv('/kaggle/working/order_products__train.csv')
order_products_prior = pd.read_csv('/kaggle/working/order_products__prior.csv')
products = pd.read_csv('/kaggle/working/products.csv')
aisles = pd.read_csv('/kaggle/working/aisles.csv')
departments = pd.read_csv('/kaggle/working/departments.csv')

In [ ]:
# For each dataset, we convert string to category. orders_products, user_reorder product_reorder users_products
aisles['aisle'] = aisles['aisle'].astype('category')
departments['department'] = departments['department'].astype('category')
orders['eval_set'] = orders['eval_set'].astype('category')
products['product_name'] = products['product_name'].astype('category')

In [ ]:
#Merge orders and order_products_prior by order_id
orders_products = pd.merge(orders,order_products_prior, on='order_id', how='inner')
#Count total orders per user
user = orders_products.groupby('user_id')['order_number'].max().to_frame('u_total_orders')
user = user.reset_index()
user.head()

In [ ]:
#caculate reorder ratio of user by reorders / all orders for each user.
user_reorder = orders_products.groupby('user_id')['reordered'].mean().to_frame('user_reordered_ratio')
user_reorder = user_reorder.reset_index()
user_reorder.head()
user = user.merge(user_reorder, on='user_id', how='left')

user.head()

In [ ]:
#Count total orders per product id
products = orders_products.groupby('product_id')['order_id'].count().to_frame('p_total_purchases')
products = products.reset_index()

#remove produdcts with small amount of purchases
product_reorder = orders_products.groupby('product_id').filter(lambda x: x.shape[0] >260)
#find reorder ratio of products
product_reorder = product_reorder.groupby('product_id')['reordered'].mean().to_frame('product_reorder_ratio')
product_reorder = product_reorder.reset_index()
product_reorder.head()

In [ ]:
#Merge the product with its reorder rate by id
products = pd.merge(products, product_reorder, on='product_id', how='left')

#replace null values with 0
products['product_reorder_ratio'] = products['product_reorder_ratio'].fillna(value=0)
products.head()

In [ ]:
# Create groups for each unique of user and product and count total amount purchased 
users_products = orders_products.groupby(['user_id', 'product_id'])['order_id'].count().to_frame('users_products_total_bought')

users_products = users_products.reset_index()
users_products.head()

In [ ]:
#Find how often a user would purchase  after first purchas 
total_times = orders_products.groupby(['user_id', 'product_id'])[['order_id']].count()
total_times.columns = ['Times_Bought']
total_times.head()

In [ ]:
#total orders of a user no matter what product is included
total_orders = orders_products.groupby('user_id')['order_number'].max().to_frame('total_orders')
total_orders.head()
#get first order of each unique user and product
first_order_no = orders_products.groupby(['user_id', 'product_id'])['order_number'].min().to_frame('first_order_number')
first_order_no  = first_order_no.reset_index()
first_order_no.head()

In [ ]:
span = pd.merge(total_orders, first_order_no, on='user_id', how='right')

# we add one to compensate for first_order_number
span['Order_Range_D'] = span.total_orders - span.first_order_number + 1
span.head()

In [ ]:
#merge total_times and span based on user and product id
users_products_ratio = pd.merge(total_times, span, on=['user_id', 'product_id'], how='left')
#get ratio based on times and new order range
users_products_ratio['users_products_reorder_ratio'] = users_products_ratio.Times_Bought / users_products_ratio.Order_Range_D
#clean up dataframe
users_products_ratio = users_products_ratio.drop(['Times_Bought', 'total_orders', 'first_order_number', 'Order_Range_D'], axis=1)
users_products_ratio.head()

In [ ]:
#merge based on unique user,product ids
users_products = pd.merge(users_products,users_products_ratio, on=['user_id', 'product_id'], how='left')


users_products.head()

In [ ]:
#Merge users_products features with the user features
#Store all relevant from user, products into final dataframe
data = data.merge(data,user, on='user_id', how='left')

#Merge users_products & user features (the new DataFrame) with products features
data = pd.merge(data,products, on='product_id', how='left')
data.head()

In [ ]:
del users_products_ratio
del product_reorder
del user_reorder
del [total_times, first_order_no, span]
del orders_products, user, products, _xp
gc.collect()

In [ ]:
## First approach:
# get orders with train and test, and keep only relevant variables
orders_future = orders[((orders.eval_set=='train') | (orders.eval_set=='test'))]
orders_future = orders_future[ ['user_id', 'eval_set', 'order_id'] ]
orders_future.head(10)

In [ ]:
# merge into data based on user id 
data = pd.merge(data,orders_future, on='user_id', how='left')
data.head(10)

In [ ]:
#create new dataframe with only training data
train_data = data[data.eval_set=='train']

#for the training data, add wheteher there was a reorder or not, 1 = yes 0 = no
train_data = train_data.merge(order_products_train[['product_id','order_id', 'reordered']], on=['product_id','order_id'], how='left' )

#clean all null values with 0 for not reorderd
train_data['reordered'] = train_data['reordered'].fillna(0)
train_data.head()
#create index based on id of user and products
train_data = train_data.set_index(['user_id', 'product_id'])
train_data.head()
#We remove all non-predictor variables
train_data = train_data.drop(['eval_set', 'order_id'], axis=1)
train_data.head(15)

In [ ]:
#create new dataframe with only test data
test_data = data[data.eval_set=='test']

#create index based on id of user and products
test_data = test_data.set_index(['user_id', 'product_id'])
test_data.head()
#We remove all non-predictor variables
test_data = test_data.drop(['eval_set','order_id'], axis=1)
#Check if the test_data DF, has the same number of columns as the train_data DF, excluding the response variable
test_data.head()

In [ ]:
train_data.loc[:, 'reordered'] = train_data.reordered.fillna(0)


X_train, X_val, y_train, y_val = train_test_split(train_data.drop('reordered', axis=1), train_data.reordered,
                                                    test_size=0.2, random_state=42)


d_train = xgboost.DMatrix(X_train, y_train)
xgb_params = {
    "objective"         : "reg:logistic"
    ,"eval_metric"      : "logloss"
    ,"eta"              : 0.1
    ,"max_depth"        : 6
    ,"min_child_weight" :10
    ,"gamma"            :0.70
    ,"subsample"        :0.76
    ,"colsample_bytree" :0.95
    ,"alpha"            :2e-05
    ,"lambda"           :10
}
watchlist= [(d_train, "train")]
bst = xgboost.train(params=xgb_params, dtrain=d_train, num_boost_round=80, evals=watchlist, verbose_eval=10)
xgboost.plot_importance(bst)

In [ ]:
#use knn with 10 neighbors
kNN = KNeighborsClassifier(n_neighbors=10)
kNN.fit(X_train, y_train)

In [ ]:
knn_pred = kNN.predict(X_val)

In [ ]:
print("Accuracy:", accuracy_score(y_val, knn_pred))
knn_pred

In [ ]:
del [X_train, X_val, y_train, y_val]
gc.collect()

In [ ]:
pred = xgboost.DMatrix(test_data)

prediction = test_data.reset_index()
prediction = prediction[['product_id', 'user_id']]

prediction["reordered"] = (bst.predict(pred) > 0.15).astype(int)


In [ ]:
orders_test = orders[orders.eval_set=='test']
prediction = prediction.merge(orders_test[["user_id", "order_id"]], on='user_id', how='left')
prediction.head()

In [ ]:
#Save the prediction in a new column intest_data 
del orders
del orders_test
gc.collect()

In [ ]:
prediction['product_id'] = prediction.product_id.astype(int)
prediction = prediction.drop('user_id', axis=1)
gc.collect()

In [ ]:
#create submisssion for kaggle
d = dict()
for row in prediction.itertuples():
    if row.reordered == 1:
        try:
            d[row.order_id] += ' ' + str(row.product_id)
        except:
            d[row.order_id] = str(row.product_id)

for order in prediction.order_id:
    if order not in d:
        d[order] = 'None'
        
gc.collect()

In [ ]:
#Convert the dictionary into a DataFrame
submisssion = pd.DataFrame.from_dict(d, orient='index')

#Reset index
submisssion.reset_index(inplace=True)
#Set column names
submisssion.columns = ['order_id', 'products']

submisssion.head()

In [ ]:
#Check if sub file has 75000 predictions
#submisssion.shape[0]
#print(submisssion.shape[0]==75000)

In [ ]:
submisssion.to_csv('submisssion.csv', index=False)